In [19]:
# %load run_motion.py
import os
import numpy as np
import cv2
from scipy.interpolate import RectBivariateSpline
from skimage.filters import apply_hysteresis_threshold
import time
from matplotlib import pyplot as plt

def lucas_kanade_affine(img1, img2, p, Gx, Gy):
    ### START CODE HERE ###
    # [Caution] From now on, you can only use numpy and 
    # RectBivariateSpline. Never use OpenCV.
    temp = img1.copy()
    img  = img2.copy()/255.0
    Gx_ = Gx.copy()/255.0
    Gy_ = Gy.copy()/255.0

    height, width = temp.shape
    x = np.arange(0,width,1)
    y = np.arange(0,height,1)
    img_spline = RectBivariateSpline(y, x, img)
    Gx_spline  = RectBivariateSpline(y, x, Gx_)
    Gy_spline  = RectBivariateSpline(y, x, Gy_)
    gxmean = np.mean(abs(Gx))
    gymean = np.mean(abs(Gy))
    
    
    H = np.zeros((6,6))
    error_sum = np.zeros((6,1))
    
    for y_ in y:
        for x_ in x:
            wx = (1 + p[0,0])*x_ + p[2,0]*y_ + p[4,0]
            wy = p[1,0]*x_ + (1 + p[3,0])*y_ + p[5,0]
            if (wx >= 0 and wy >= 0 and wx <= width-1 and wy <= height-1):
                gx = float(Gx_spline.ev(wy,wx)*255.0)
                gy = float(Gy_spline.ev(wy,wx)*255.0)
                if (abs(gx) < gxmean and abs(gy) < gymean): 
                    continue
                value = float(img_spline.ev(wy,wx)*255.0)
                if (value < 0 or value > 255):
                    continue
                error = temp[y_,x_] - value
                grad = np.array([[gx*x_],
                                 [gy*x_],
                                 [gx*y_],
                                 [gy*y_],
                                 [gx],
                                 [gy]])
                error_sum = error_sum + grad*error
                H = H + grad@grad.T
    inv_H = np.linalg.pinv(H)
    dp = inv_H@error_sum    
    ### END CODE HERE ###
    return dp

def subtract_dominant_motion(img1, img2):
    Gx = cv2.Sobel(I, cv2.CV_64F, 1, 0, ksize = 5) # do not modify this
    Gy = cv2.Sobel(I, cv2.CV_64F, 0, 1, ksize = 5) # do not modify this
    th_hi = 0.5*256 # you can modify this
    th_lo = 0.2*256 # you can modify this

    ### START CODE HERE ###
    # [Caution] From now on, you can only use numpy and 
    # RectBivariateSpline. Never use OpenCV.
    P = np.zeros((6,1))
    
    temp = img1.copy()
    img  = img2.copy()
    dp = lucas_kanade_affine(temp, img, P, Gx, Gy)
    P = P + dp
    first = np.linalg.norm(dp)
    print(first)
    dp = lucas_kanade_affine(temp, img, P, Gx, Gy)
    second = np.linalg.norm(dp)
    print(abs(first-second))

    height, width = img.shape    
    x = np.arange(0,width,1)
    y = np.arange(0,height,1)
    temp_norm = temp/255.0
    temp_spline = RectBivariateSpline(y, x, temp_norm)
    
    moving_image = np.zeros(img2.shape)
    for y_ in y:
        for x_ in x:
            wx = ((x_-P[4,0])*(1+P[3,0])-(y_-P[5,0])*P[2,0])/(-P[1,0]*P[2,0]+(1+P[0,0])*(1+P[3,0]))
            wy = ((x_-P[4,0])*P[1,0]-(y_-P[5,0])*(1+P[0,0]))/(P[1,0]*P[2,0]-(1+P[0,0])*(1+P[3,0]))
            if (wx >= 0 and wy >= 0 and wx <= width-1 and wy <= height-1):
                value = float(temp_spline.ev(wy,wx)*255)
                if (value >= 0 and value <= 255):
                    moving_image[y_][x_] = abs(img2[y_][x_] - value)
    
    ### END CODE HERE ###

    hyst = apply_hysteresis_threshold(moving_image, th_lo, th_hi)
    return hyst

if __name__ == "__main__":
    start = time.time()
    data_dir = 'data'
    video_path = 'motion.mp4'
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(video_path, fourcc, 150/20, (636, 318))
    tmp_path = os.path.join(data_dir, "organized-{}.jpg".format(0))
    T = cv2.cvtColor(cv2.imread(tmp_path), cv2.COLOR_BGR2GRAY)

    for i in range(0, 50):
        img_path = os.path.join(data_dir, "organized-{}.jpg".format(i))
        I = cv2.cvtColor(cv2.imread(img_path), cv2.COLOR_BGR2GRAY)
        clone = I.copy()
        moving_img = subtract_dominant_motion(T, I)
        clone = cv2.cvtColor(clone, cv2.COLOR_GRAY2BGR)
        clone[moving_img, 2] = 522
        out.write(clone)
        T = I
    out.release()
    print((time.time()-start)/60)
    

5.967944277454606e-19
0.0
0.012889586246297989
8.429995855378097e-06
0.007644644748060568
4.038616894637241e-05
0.005780307751544401
2.6010639198599354e-05
0.009384653996945781
1.649280132054627e-05
0.015435953952306755
8.383594856784318e-06
0.009243331767463264
1.138552538589195e-05
0.00802937576672519
0.00011461391712060467
0.007711181867445128
0.00012569786459376998
0.007232831730949599
0.00013535572088328122
0.012284319486431992
1.2874457854562815e-05
0.0060956541651357886
7.210218993959454e-05
0.0038720242142535944
1.4115423521534123e-05
0.012538228429855509
0.00011443726974232071
0.0038936341509506877
1.0813862334202451e-05
0.013250309750575026
5.0300759922575905e-05
0.010732042287435234
2.7855175177903674e-05
0.009955484034705295
6.35003902331073e-05
0.00875161367787131
4.7504119311134185e-05
0.011229990131044167
2.1970611994040143e-05
0.011025855209841999
1.406913578156678e-05
0.008516360352233496
4.92880234391653e-05
0.007494463012860224
1.89894197869106e-05
0.0118445838173073